In [30]:
import pandas as pd


In [31]:
pd.ExcelFile("../data/sh_isac_2025.xls").sheet_names


['Índice',
 'Cuadro 1',
 'Cuadro 2.1',
 'Cuadro 2.2',
 'Cuadro 2.3',
 'Cuadro 3.1',
 'Cuadro 3.2',
 'Cuadro 4.1',
 'Cuadro 4.2',
 'Cuadro 5',
 'Cuadro 6.1',
 'Cuadro 6,2',
 'Cuadro 6.3',
 'Cuadro 7.1',
 'Cuadro 7.2',
 'Cuadro 7.3',
 'Cuadro 7.4',
 'Cuadro 7.5',
 'Cuadro 7.6',
 'Cuadro 7.7']

In [23]:
df_raw = pd.read_excel(
    "../data/sh_isac_2025.xls",
    sheet_name=1,
    header=None
)

df_raw.head(20)


,0,1,2,3,4,5,6,7,8,9,10
0,Cuadro 1. Indicador sintético de la actividad ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Período,NaN,Serie original,NaN,NaN,NaN,Serie desestacionalizada (1),NaN,NaN,Serie tendencia-ciclo,NaN
3,NaN,NaN,ISAC nivel general,Variación porcentual,NaN,NaN,ISAC nivel general,Variación porcentual respecto al mes anterior,NaN,ISAC nivel general,Variación porcentual respecto al mes anterior
4,NaN,NaN,NaN,respecto al mismo mes del año anterior,acumulada del año respecto a igual acumulado d...,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,Números índice,%,NaN,NaN,Números índice,%,NaN,Números índice,%
6,2012,Enero,169.90712,///,///,NaN,181.692949,///,NaN,175.748264,///
7,NaN,Febrero,151.766779,///,///,NaN,172.529606,-5.043312,NaN,175.076517,-0.382221
8,NaN,Marzo,188.20224,///,///,NaN,181.927216,5.446955,NaN,174.308709,-0.438556
9,NaN,Abril,164.243978,///,///,NaN,172.581638,-5.136987,NaN,173.505746,-0.460656


In [24]:
df = df_raw.iloc[6:, [0, 1, 2]]
df.columns = ["year", "month", "isac"]

# año solo aparece en enero
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["year"] = df["year"].ffill()


In [25]:
valid_months = {
    "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
    "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
}

df = df[df["month"].isin(valid_months)]
df["month"].value_counts().sort_index()



Abril         14
Agosto        14
Diciembre     13
Enero         14
Febrero       14
Julio         14
Junio         14
Marzo         14
Mayo          14
Noviembre     13
Octubre       14
Septiembre    14
Name: month, dtype: int64

In [26]:
month_map = {
    "Enero": 1, "Febrero": 2, "Marzo": 3, "Abril": 4,
    "Mayo": 5, "Junio": 6, "Julio": 7, "Agosto": 8,
    "Septiembre": 9, "Octubre": 10, "Noviembre": 11, "Diciembre": 12
}

df["month_num"] = df["month"].map(month_map)

df["date"] = pd.to_datetime(
    df["year"].astype(int).astype(str) + "-" +
    df["month_num"].astype(int).astype(str) + "-01"
)

df = df[["date", "isac"]].sort_values("date")


In [27]:
df["isac"] = pd.to_numeric(df["isac"], errors="coerce")
df = df.dropna()

df = df.drop_duplicates(subset=["date"], keep="last")

df["isac_yoy"] = df["isac"].pct_change(12) * 100
df.tail(12)


,date,isac,isac_yoy
172,2024-11-01,150.032805,-24.008908
173,2024-12-01,133.448214,-10.209079
175,2025-01-01,132.989787,-1.370485
176,2025-02-01,126.824139,3.723645
177,2025-03-01,133.526512,15.750312
178,2025-04-01,145.999202,25.940057
179,2025-05-01,141.771015,9.369204
180,2025-06-01,141.172254,14.056358
181,2025-07-01,154.412037,1.498121
182,2025-08-01,154.337425,0.370704


In [28]:
df["date"].diff().value_counts().head()



31 days    96
30 days    55
28 days    10
29 days     4
Name: date, dtype: int64

In [29]:
df.to_csv("../data/isac_yoy.csv", index=False)   # en ISAC